In [ ]:
# Use the "c-TF-IDF" strategy with a threshold
new_topics = model.reduce_outliers(df_tweets_preprocessed['text_preprocessed'], topics, strategy="c-tf-idf")
model.update_topics(df_tweets_preprocessed['text_preprocessed'], topics=new_topics)

In [ ]:
# Assign the updated topics to the DataFrame
# df_tweets_preprocessed['updated_topics'] = new_topics

In [ ]:
# freq = model.get_topic_info()
# freq

In [ ]:
# import sys

# # Define a file path where you want to save the output
# output_file_path = 'output.txt'

# # Create a file for writing the output
# with open(output_file_path, 'w') as output_file:
#     # Redirect sys.stdout to the output file
#     sys.stdout = output_file

#     best_topics = [1, 5, 10, 17, 21]
#     for i in best_topics:
#         df_tweets_topic_i = df_tweets_preprocessed[df_tweets_preprocessed['updated_topics'] == i]
#         df_tweets_topic_i = pd.merge(
#             df_tweets_topic_i,
#             freq,
#             how="left",
#             left_on=['updated_topics'],
#             right_on=['Topic']
#         )
#         name = freq[freq['Topic'] == i]['Name']
#         print(f"Topic: {name}")
#         print(df_tweets_topic_i['text_original'].head(100))

# # Reset sys.stdout to its original value to print to the console again
# sys.stdout = sys.__stdout__

In [ ]:
# #Which topics are you interested in?

# # 1_twitch_sponsored_stream_game: STREAMING, GAMING - 2
# # 4_climate_energy_learn_unep	: ENVIRONMENT & ENERGY SUSTAINABILITY - 4
# # 9_health_research_cancer_dr: HEALTH AND CANCER RESEARCH & INNOVATION - 10
# # 17_ai_machinelearning_future_cc: ARTIFICIAL INTELLGIENCE RESEARCH & INNOVATION - 17
# # 22_women_un_amp_learn	: HUMAN RIGHTS, EQUALITY, AND WOMEN EMPOWERMENT - 22
# custom_name_list = {1: "Tesla Giveaway",
#                     2: "Streaming & Gaming",
#                     3: "Optimus",
#                     4: "Nike Ads",
#                     5: "Crypto" ,
#                     6: "Environment & Energy Sustainability",
#                     7: "Amazon Deals",
#                     8: "Youtube Education",
#                     9: "Spam",
#                     10: "Blockchain Spam",
#                     11: "Health and Cancer Research & Innovation",
#                     12: "Online Deals",
#                     13: "Ford Car Deals",
#                   # 14: "LinkedIn Events",
#                     15: "Iphone Network",
#                     16: "Memes",
#                     17: "Media",
#                     18: "Artificial Intelligence Research & Innovation",
#                     19: "News",
#                     20: "Ebay Sports Trading Cards",
#                     21: "Shopping",
#                     22: "Human Rights & Equality"}

In [ ]:
model = BERTopic.load("/content/drive/MyDrive/Colab_Notebooks/My_Models")
freq = model.get_topic_info()
freq

In [ ]:
df_tweets_pp_temp = pd.concat([df_tweets_preprocessed, pd.Series(model.topics_)],
                 axis=1)

print(df_tweets_pp_temp.info())

In [ ]:
df_tweets_pp_final = df_tweets_pp_temp.merge(freq,
                                            how='left',
                                            left_on=0,
                                            right_on=['Topic'])

In [ ]:
best_topics = [1, 5, 10, 17, 21]
df_tweets_pp_final = df_tweets_pp_final[df_tweets_pp_final[0].isin(best_topics)]

In [ ]:
df_tweets_pp_final = df_tweets_pp_final.merge(users_df,
                                               on=['author_id'],
                                               how='left')

In [ ]:
# Assuming your DataFrame is named 'df'
df_tweets_pp_final.rename(columns={'0': 'topic'}, inplace=True)

In [ ]:
import ast
df_tweets_pp_final['tweet_metrics'] = df_tweets_pp_final['tweet_metrics'].apply(ast.literal_eval)

In [ ]:
df_tweets_pp_final['retweet_count']     = df_tweets_pp_final['tweet_metrics'].apply(lambda x: x.get('retweet_count', 0))
df_tweets_pp_final['reply_count']       = df_tweets_pp_final['tweet_metrics'].apply(lambda x: x.get('reply_count', 0))
df_tweets_pp_final['like_count']        = df_tweets_pp_final['tweet_metrics'].apply(lambda x: x.get('like_count', 0))
df_tweets_pp_final['impression_count']  = df_tweets_pp_final['tweet_metrics'].apply(lambda x: x.get('impression_count', 0))

In [ ]:
# Group by specified columns and aggregate metrics
df_tweets_grouped = df_tweets_pp_final.groupby(['author_id', 'username', 'followers_count', 'following_count', 'tweet_count', 'CustomName']).agg(
    total_tweets=('tweet_id', 'count'),
    total_retweets=('retweet_count', 'sum'),
    total_replies=('reply_count', 'sum'),
    total_likes=('like_count', 'sum'),
    total_impressions=('impression_count', 'sum')
).reset_index()

# Calculate average likes, impressions, and replies per tweet
df_tweets_grouped['avg_likes_per_tweet']        = df_tweets_grouped['total_likes'] / df_tweets_grouped['total_tweets']
df_tweets_grouped['avg_impressions_per_tweet']  = df_tweets_grouped['total_impressions'] / df_tweets_grouped['total_tweets']
df_tweets_grouped['avg_replies_per_tweet']      = df_tweets_grouped['total_replies'] / df_tweets_grouped['total_tweets']

In [ ]:
# Define weights for each criterion
weights = {
    'Followers': 0.2,
    'Tweets': 0.2,
    'Avg_Likes_Per_Tweet': 0.25,
    'Avg_Impressions_Per_Tweet': 0.25,
    'Avg_Replies_Per_Tweet': 0.10
}

# Calculate a weighted score for each user
df_tweets_grouped['user_score'] = (
    weights['Followers'] * df_tweets_grouped['followers_count'] +
    weights['Tweets'] * df_tweets_grouped['total_tweets'] +
    weights['Avg_Likes_Per_Tweet'] * df_tweets_grouped['avg_likes_per_tweet'] +
    weights['Avg_Impressions_Per_Tweet'] * df_tweets_grouped['avg_impressions_per_tweet'] +
    weights['Avg_Replies_Per_Tweet'] * df_tweets_grouped['avg_replies_per_tweet']
)

# Sort users by topic and user_score
sorted_grouped = df_tweets_grouped.sort_values(by=['CustomName', 'user_score'], ascending=[True, False])

# Get the top 5 users per topic
top_users_per_topic = sorted_grouped.groupby('CustomName').head(5)

In [ ]:
sorted_grouped[(sorted_grouped['total_tweets'] >= 3)]